# Tutorial FFT 3D parallel (MPI): Domain decomposition
We have seen that FluidFFT provides a unified framework for different parallelized FFT 3D libraries, viz. FFTW (with MPI), P3DFFT, and PFFT.

In this tutorial, we will look into how these libraries perform domain decomposition, and thereby try to balance the load evenly. Understanding how this is done is important to plan the discretization (i.e. shape of the arrays).

Always remember:

> "FFTW is best at handling sizes of the form $2^a \times 3^b \times 5^c \times 7^d \times 11^e \times 13^f$, where $e+f$ is either 0 or 1, and the other exponents are arbitrary. Other sizes are computed by means of a slow, general-purpose routine (which nevertheless retains $O(n \log n)$ performance, even for prime sizes). (It is possible to customize FFTW for different array sizes. See Section [Installation and Customization](http://www.fftw.org/fftw2_doc/fftw_6.html#SEC66), for more information.) Transforms whose sizes are powers of 2 are especially fast."
>
> Source: http://www.fftw.org/fftw2_doc/fftw_3.html

Just like, before we start an parallelized IPython/Jupyter session with `ipcluster start -n 4 --engines=MPIEngineSetLauncher`.

In [ ]:
import ipyparallel as ipp
rc = ipp.Client()
dview = rc[:]

We start by importing all the functions that we need

In [2]:
%%px
from fluiddyn.util.info import _print_dict
from fluidfft.fft3d import get_classes_mpi
from fluiddyn.util.mpi import rank

In [3]:
%%px
dict_classes = get_classes_mpi()

The function `get_classes_mpi` creates a dictionary of all available FFT classes.

In [4]:
%%px  --targets 1
_print_dict(dict_classes)

 - fft3d.mpi_with_fftw1d           : <class 'fluidfft.fft3d.mpi_with_fftw1d.FFT3DMPIWithFFTW1D'>
 - fft3d.mpi_with_fftwmpi3d        : <class 'fluidfft.fft3d.mpi_with_fftwmpi3d.FFT3DMPIWithFFTWMPI3D'>
 - fft3d.mpi_with_p3dfft           : <class 'fluidfft.fft3d.mpi_with_p3dfft.FFT3DMPIWithP3DFFT'>
 - fft3d.mpi_with_pfft             : <class 'fluidfft.fft3d.mpi_with_pfft.FFT3DMPIWithPFFT'>


We now chose a small shape for the purpose of this tutorial, compatible with FFTW requirements: say $20 \times 12 \times 8$, and instantiate FFT operators (or objects) of the above classes. Let us compose a nifty function which takes an FFT class as the argument, instantiates it with the shape and prints the information we seek.

In [5]:
%%px
def fft_info(cls):
    """Instanitate and display array shapes"""
    
    opfft = cls(20, 12, 8)
    if rank == 0:
        print('get_dimX_K :'.rjust(35), opfft.get_dimX_K())
    
    print(
        'Global physical shape :'.rjust(35), opfft.get_shapeX_seq(),
        'Local physical shape :'.rjust(35),  opfft.get_shapeX_loc())
    print(
        'Global FFT shape :'.rjust(35), opfft.get_shapeK_seq(),
        'Local FFT shape :'.rjust(35),  opfft.get_shapeK_loc())
    del opfft

## fft3d.mpi_with_fftw1d

In [6]:
%%px
fft_info(dict_classes['fft3d.mpi_with_fftw1d'])

[stdout:0] 
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (4, 12, 20)                   Local FFT shape : (1, 12, 20)
[stdout:1] 
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (4, 12, 20)                   Local FFT shape : (1, 12, 20)
[stdout:2] 
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (4, 12, 20)                   Local FFT shape : (1, 12, 20)
[stdout:3] 
                       get_dimX_K : (2, 1, 0)
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (4, 12, 20)                   Local FFT shape : (1, 12, 20)


## fft3d.mpi_with_fftwmpi3d

In [7]:
%%px
fft_info(dict_classes['fft3d.mpi_with_fftwmpi3d'])

[stdout:0] 
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (12, 20, 5)                   Local FFT shape : (3, 20, 5)
[stdout:1] 
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (12, 20, 5)                   Local FFT shape : (3, 20, 5)
[stdout:2] 
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (12, 20, 5)                   Local FFT shape : (3, 20, 5)
[stdout:3] 
                       get_dimX_K : (1, 0, 2)
            Global physical shape : (20, 12, 8)              Local physical shape : (5, 12, 8)
                 Global FFT shape : (12, 20, 5)                   Local FFT shape : (3, 20, 5)


## fft3d.mpi_with_p3dfft

In [8]:
%%px
fft_info(dict_classes['fft3d.mpi_with_p3dfft'])

[stdout:0] 
            Global physical shape : (20, 12, 8)              Local physical shape : (20, 6, 4)
                 Global FFT shape : (20, 12, 8)                   Local FFT shape : (5, 6, 8)
[stdout:1] 
            Global physical shape : (20, 12, 8)              Local physical shape : (20, 6, 4)
                 Global FFT shape : (20, 12, 8)                   Local FFT shape : (6, 6, 8)
[stdout:2] 
            Global physical shape : (20, 12, 8)              Local physical shape : (20, 6, 4)
                 Global FFT shape : (20, 12, 8)                   Local FFT shape : (6, 6, 8)
[stdout:3] 
                       get_dimX_K : (0, 1, 2)
            Global physical shape : (20, 12, 8)              Local physical shape : (20, 6, 4)
                 Global FFT shape : (20, 12, 8)                   Local FFT shape : (5, 6, 8)


## fft3d.mpi_with_pfft

In [9]:
%%px
fft_info(dict_classes['fft3d.mpi_with_pfft'])

[stdout:0] 
            Global physical shape : (20, 12, 8)              Local physical shape : (10, 6, 8)
                 Global FFT shape : (12, 5, 20)                   Local FFT shape : (6, 3, 20)
[stdout:1] 
            Global physical shape : (20, 12, 8)              Local physical shape : (10, 6, 8)
                 Global FFT shape : (12, 5, 20)                   Local FFT shape : (6, 2, 20)
[stdout:2] 
            Global physical shape : (20, 12, 8)              Local physical shape : (10, 6, 8)
                 Global FFT shape : (12, 5, 20)                   Local FFT shape : (6, 2, 20)
[stdout:3] 
                       get_dimX_K : (1, 2, 0)
            Global physical shape : (20, 12, 8)              Local physical shape : (10, 6, 8)
                 Global FFT shape : (12, 5, 20)                   Local FFT shape : (6, 3, 20)


## Summary
We have only looked at the default options of the FFT classes. It is possible to transpose and customize array ordering. Different approaches are adopted by different FFT libraries both in terms of array ordering and and distributing processes.

For a physical array ordered like $(n_0, n_1, n_2)$ and with $n_p$ MPI processes


$$\newcommand{nphalf}[0]{\frac{n_p}{2}}$$


|Method                  |  FFT array order       |  Physical array process grid |  FFT array process grid |
|------------------------|------------------------|------------------------------|-------------------------|
|fft3d.mpi_with_fftw1d   |    $(2, 1, 0)$         |$(n_p, 1, 1)$                 |$(n_p, 1, 1)$            |
|fft3d.mpi_with_fftwmpi3d|    $(1, 0, 2)$         |$(n_p, 1, 1)$                 |$(n_p, 1, 1)$            |
|fft3d.mpi_with_p3dfft   |    $(0, 1, 2)$         |$(1, \nphalf, \nphalf)$       |$(\nphalf, \nphalf, 1)$  |
|fft3d.mpi_with_pfft     |    $(1, 2, 0)$         |$(\nphalf, \nphalf, 1)$       |$(\nphalf, \nphalf, 1)$  |


Note that FFTW methods distributes processes only over one index, i.e. splits the global array into **slabs**. On the other hand P3DFFT and PFFT distributes processes over 2 indices, i.e. splitting the global array in 2 dimensions (also known as **pencil decomposition**).